In [1]:
print("ok")

ok


In [2]:
# Install some required packages

!pip install pypdf2
!pip install google-cloud-aiplatform
!pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00


In [3]:
from google.cloud import storage
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import PyPDF2

import re
import os
import random
import json
import uuid


In [4]:
%ls

stats.pdf


In [5]:
# Initialize some variable

# project="your_GCP_project_id"
location="us-central1"

pdf_path="stats.pdf"
bucket_name = "stats-content"
embed_file_path = "stats_embeddings.json"
sentence_file_path = "stats_sentences.json"
index_name="stats_index"



In [6]:
# helper

def extract_sentences_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            if page.extract_text() is not None:
                text += page.extract_text() + " "
    sentences = [sentence.strip() for sentence in text.split('. ') if sentence.strip()]
    return sentences

In [7]:
def generate_text_embeddings(sentences) -> list:
  # aiplatform.init(project=project,location=location)
  model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
  embeddings = model.get_embeddings(sentences)
  vectors = [embedding.values for embedding in embeddings]
  return vectors

In [8]:
def generate_and_save_embeddings(pdf_path, sentence_file_path, embed_file_path):
    def clean_text(text):
        cleaned_text = re.sub(r'\u2022', '', text)  # Remove bullet points
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra whitespaces and strip
        return cleaned_text

    sentences = extract_sentences_from_pdf(pdf_path)
    if sentences:
        embeddings = generate_text_embeddings(sentences)

        with open(embed_file_path, 'w') as embed_file, open(sentence_file_path, 'w') as sentence_file:
            for sentence, embedding in zip(sentences, embeddings):
                cleaned_sentence = clean_text(sentence)
                id = str(uuid.uuid4())

                embed_item = {"id": id, "embedding": embedding}
                sentence_item = {"id": id, "sentence": cleaned_sentence}

                json.dump(sentence_item, sentence_file)
                sentence_file.write('\n')
                json.dump(embed_item, embed_file)
                embed_file.write('\n')

In [9]:
def upload_file(bucket_name,file_path):
    storage_client = storage.Client()
    bucket = storage_client.create_bucket(bucket_name,location=location)
    blob = bucket.blob(file_path)
    blob.upload_from_filename(file_path)

In [10]:
def create_vector_index(bucket_name, index_name):
    lakeside_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name = index_name,
    contents_delta_uri = "gs://"+bucket_name,
    dimensions = 768,
    approximate_neighbors_count = 10,
    )

    lakeside_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name = index_name,
    public_endpoint_enabled = True
    )

    lakeside_index_endpoint.deploy_index(
    index = lakeside_index, deployed_index_id = index_name
    )

In [11]:
generate_and_save_embeddings(pdf_path,sentence_file_path,embed_file_path)
upload_file(bucket_name,sentence_file_path)

In [12]:
create_vector_index(bucket_name, index_name)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/1060807096482/locations/us-central1/indexes/3439259177543794688/operations/7478939149387431936
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/1060807096482/locations/us-central1/indexes/3439259177543794688
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/1060807096482/locations/us-central1/indexes/3439259177543794688')
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngi

ResourceExhausted: 429 The following quotas are exceeded: MatchingEngineDeployedIndexNodes 8: The following quotas are exceeded: MatchingEngineDeployedIndexNodes

In [13]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import json
import os

In [ ]:
# project=”YOUR_GCP_PROJECT”
location="us-central1"
sentence_file_path = "stats_sentences.json"
index_name="stats_index" #Get this from the console or the previous step

In [15]:
# aiplatform.init(project=project,location=location)
# vertexai.init()
model = GenerativeModel("gemini-pro")
lakeside_index_ep = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name="5030296483400777728")

In [16]:
def generate_text_embeddings(sentences) -> list:
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    embeddings = model.get_embeddings(sentences)
    vectors = [embedding.values for embedding in embeddings]
    return vectors


def generate_context(ids,data):
    concatenated_names = ''
    for id in ids:
        for entry in data:
            if entry['id'] == id:
                concatenated_names += entry['sentence'] + "\n"
    return concatenated_names.strip()



def load_file(sentence_file_path):
  data = []
  with open(sentence_file_path,'r') as f:
    for line in f:
      entry = json.loads(line)
      data.append(entry)
  return data





In [17]:
data=load_file(sentence_file_path)
data

[{'id': '9f6055cb-fd9f-4fb4-a8e6-f8570c267098',
  'sentence': 'Importance and the use of correlation in Statistics Introduction Correlation is a statistical measure that expresses the extent to which two variables are linearly related. It is a common tool for describing simple relationships without making a statement about cause and effect. Correlation coefficients range from -1 to 1, with a value of 0 indicating no linear relationship between the two variables, a value of 1 indicating a perfect positive linear relationship, and a value of -1 indicating a perfect negative linear relationship. Correlation is important in statistics because it can be used to 1. Identify relationships between variables: Correlation can be used to identify whether there is a relationship between two variables, and if so, whether the relationship is positive or negative. This information can be useful for understanding the relationships between different factors in a complex system. 2. Make predictions: If 

In [18]:
query=["what is correlation?"]

qry_emb=generate_text_embeddings(query)

In [19]:
qry_emb

[[0.0031879430171102285,
  0.030173784121870995,
  -0.046163856983184814,
  -0.010091220960021019,
  -0.026898862794041634,
  -0.05871526896953583,
  0.022807922214269638,
  0.02700706385076046,
  -0.021587025374174118,
  0.03865209221839905,
  -0.014295038767158985,
  -0.04151195287704468,
  0.035884130746126175,
  -5.352220614440739e-05,
  -0.039067499339580536,
  0.007143754046410322,
  -0.02639135904610157,
  -0.015075067058205605,
  0.043786194175481796,
  -0.019433336332440376,
  -0.03991476446390152,
  -0.060704465955495834,
  -0.019392838701605797,
  -0.01959632895886898,
  -0.019830776378512383,
  -0.10087926685810089,
  0.00258005503565073,
  0.007023989222943783,
  -0.025977298617362976,
  -0.028022313490509987,
  0.03841837868094444,
  0.01058691181242466,
  0.017269540578126907,
  0.003389855148270726,
  -0.024171901866793633,
  0.040811534970998764,
  0.016141369938850403,
  0.026479193940758705,
  -0.03963363170623779,
  -0.013527167029678822,
  0.03784491494297981,
  -0

In [20]:
response = lakeside_index_ep.find_neighbors(
    deployed_index_id = index_name,
    queries = [qry_emb[0]],
    num_neighbors = 10
)

ServiceUnavailable: 503 DNS resolution failed for 1645167058.us-central1-1060807096482.vdb.vertexai.goog:443: C-ares status is not ARES_SUCCESS qtype=A name=1645167058.us-central1-1060807096482.vdb.vertexai.goog is_balancer=0: Domain name not found

In [ ]:


matching_ids = [neighbor.id for sublist in response for neighbor in sublist]

context = generate_context(matching_ids,data)
prompt=f"Based on the context delimited in backticks, answer the query. ```{context}``` {query}"

chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)